In [8]:
import pandas as pd

In [9]:
intrpr_df = pd.read_csv("../data/entry-matching-P0ACQ4.tsv", sep="\t")
intrpr_df["range"] = intrpr_df.apply(lambda r: (int(r.Matches.split("..")[0]), int(r.Matches.split("..")[1])), axis=1)
intrpr_df["start pos"] = intrpr_df.apply(lambda r: r["range"][0], axis=1)
intrpr_df["end pos"] = intrpr_df.apply(lambda r: r["range"][1], axis=1)
intrpr_df

,Accession,Name,Source Database,Type,Integrated Into,Integrated Signatures,GO Terms,Protein Accession,Protein Length,Matches,range,start pos,end pos
0,cd08411,The C-terminal substrate-binding domain of the...,cdd,domain,NaN,NaN,NaN,p0acq4,305,91..289,"(91, 289)",91,289
1,G3DSA:1.10.10.10,Winged helix-like DNA-binding domain superfami...,cathgene3d,homologous_superfamily,IPR036388,NaN,NaN,p0acq4,305,1..84,"(1, 84)",1,84
2,G3DSA:3.40.190.10,Periplasmic binding protein-like II,cathgene3d,homologous_superfamily,NaN,NaN,NaN,p0acq4,305,91..163,"(91, 163)",91,163
3,G3DSA:3.40.190.10,Periplasmic binding protein-like II,cathgene3d,homologous_superfamily,NaN,NaN,NaN,p0acq4,305,268..284,"(268, 284)",268,284
4,G3DSA:3.40.190.10,Periplasmic binding protein-like II,cathgene3d,homologous_superfamily,NaN,NaN,NaN,p0acq4,305,164..267,"(164, 267)",164,267
5,IPR000847,"Transcription regulator HTH, LysR",interpro,domain,NaN,"PS50931,PF00126,PR00039","GO:0003700,GO:0006355",p0acq4,305,1..61,"(1, 61)",1,61
6,IPR005119,"LysR, substrate-binding",interpro,domain,NaN,PF03466,NaN,p0acq4,305,89..293,"(89, 293)",89,293
7,IPR036388,Winged helix-like DNA-binding domain superfamily,interpro,homologous_superfamily,NaN,G3DSA:1.10.10.10,NaN,p0acq4,305,1..84,"(1, 84)",1,84
8,IPR036390,Winged helix DNA-binding domain superfamily,interpro,homologous_superfamily,NaN,SSF46785,NaN,p0acq4,305,1..86,"(1, 86)",1,86
9,PF00126,"Bacterial regulatory helix-turn-helix protein,...",pfam,domain,IPR000847,NaN,NaN,p0acq4,305,3..61,"(3, 61)",3,61


In [10]:
import os, sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from aledbmutil.gff import get_df_from_csv


def _update_gff_feature_description(feature, attribute):
    updated_feature = feature
    if "Note=HTH lysR-type" in attribute:
        updated_feature = "HTH domain"
    if "Disulfide bond" in feature:
        updated_feature = "Disulfide bond region"
    return updated_feature

gff_df = get_df_from_csv("../data/oxyR.gff")
gff_df = gff_df[~(gff_df["feature"].isin(["Sequence conflict", "Mutagenesis", "Beta strand", "Helix", "Turn",
#                                           "Chain"
                                         ]))]
gff_df["feature"] = gff_df.apply(lambda r: _update_gff_feature_description(r["feature"], r["attribute"]), axis=1)
gff_df

,seqname,source,feature,start,end,score,strand,frame,attribute,9
0,P0ACQ4,UniProtKB,Chain,1,305,.,.,.,ID=PRO_0000105728;Note=Hydrogen peroxide-induc...,NaN
1,P0ACQ4,UniProtKB,HTH domain,1,58,.,.,.,Note=HTH lysR-type;Ontology_term=ECO:0000255;e...,NaN
2,P0ACQ4,UniProtKB,DNA binding,18,37,.,.,.,Note=H-T-H motif;Ontology_term=ECO:0000255;evi...,NaN
3,P0ACQ4,UniProtKB,Modified residue,199,199,.,.,.,Note=Cysteine sulfenic acid (-SOH)%3B alternat...,NaN
4,P0ACQ4,UniProtKB,Modified residue,199,199,.,.,.,Note=S-glutathionyl cysteine%3B alternate;Onto...,NaN
5,P0ACQ4,UniProtKB,Modified residue,199,199,.,.,.,Note=S-nitrosocysteine%3B alternate;Ontology_t...,NaN
6,P0ACQ4,UniProtKB,Disulfide bond region,180,259,.,.,.,Ontology_term=ECO:0000269;evidence=ECO:0000269...,NaN
7,P0ACQ4,UniProtKB,Disulfide bond region,199,208,.,.,.,Note=Alternate;Ontology_term=ECO:0000269;evide...,NaN


In [11]:
def _get_barh_broken_range(r):
    return (r["start"], r["end"] - r["start"] + 1)

In [12]:
feats_df = pd.DataFrame(columns=["feature", "start", "end"])

for _, f in intrpr_df.iterrows():
    d = {"feature":f["Name"], "start":f["start pos"], "end":f["end pos"]}
    feats_df = feats_df.append(d, ignore_index=True)
    
for _, f in gff_df.iterrows():
    d = {"feature":f["feature"], "start":f["start"], "end":f["end"]}
    feats_df = feats_df.append(d, ignore_index=True)

    
# feats_df.sort_values(by=["end", "start"], ascending=False, inplace=True)
# feats_df.reset_index(drop=True, inplace=True)
feats_df["range"] = feats_df.apply(lambda r:_get_barh_broken_range(r), axis=1)  # barh_broken wants length for final value in range
feats_df

/tmp/ipykernel_4653/1734329006.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  feats_df = feats_df.append(d, ignore_index=True)
/tmp/ipykernel_4653/1734329006.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  feats_df = feats_df.append(d, ignore_index=True)
/tmp/ipykernel_4653/1734329006.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  feats_df = feats_df.append(d, ignore_index=True)
/tmp/ipykernel_4653/1734329006.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  feats_df = feats_df.append(d, ignore_index=True)
/tmp/ipykernel_4653/1734329006.py:5: FutureWarning: The frame.append method is deprecated and will be removed from p

,feature,start,end,range
0,The C-terminal substrate-binding domain of the...,91,289,"(91, 199)"
1,Winged helix-like DNA-binding domain superfami...,1,84,"(1, 84)"
2,Periplasmic binding protein-like II,91,163,"(91, 73)"
3,Periplasmic binding protein-like II,268,284,"(268, 17)"
4,Periplasmic binding protein-like II,164,267,"(164, 104)"
5,"Transcription regulator HTH, LysR",1,61,"(1, 61)"
6,"LysR, substrate-binding",89,293,"(89, 205)"
7,Winged helix-like DNA-binding domain superfamily,1,84,"(1, 84)"
8,Winged helix DNA-binding domain superfamily,1,86,"(1, 86)"
9,"Bacterial regulatory helix-turn-helix protein,...",3,61,"(3, 59)"


In [13]:
# grep "P0ACQ4" int.tab > oxyE_interfaces.tab
interface_df = pd.read_csv('../data/mutfunc/oxyR_interfaces.tab', sep='\t', header=None)
interface_df.columns = ["pdb","prot_a","prot_b","evidence","chain","aa_wt","aa_mt","pdb_pos","uniprot_pos","mut","dG_wt","dG_mt","ddG","dG_wt_sd","dG_mt_sd", "ddG_sd"]
# interface_df["NC_000913v3_pos"] = interface_df["uniprot_pos"] + 1  # Adding 1 to account for initiator methionine (start codon) within NC_000913v3 AA ref
interface_df = interface_df[interface_df.chain=='B']  # multiple chains being represented, where I believe we only need one
interface_df.sort_values(by="uniprot_pos", inplace=True)
display(len(interface_df), interface_df.head())

620

,pdb,prot_a,prot_b,evidence,chain,aa_wt,aa_mt,pdb_pos,uniprot_pos,mut,dG_wt,dG_mt,ddG,dG_wt_sd,dG_mt_sd,ddG_sd
1234,P0ACQ4-P0ACQ4-EXP-1i69-pdb1-B-0-A-0.pdb,P0ACQ4,P0ACQ4,EXP,B,H,R,94,94,HB94R,31.99580,32.37482,0.37902,0.062492,0.042879,0.094314
1236,P0ACQ4-P0ACQ4-EXP-1i69-pdb1-B-0-A-0.pdb,P0ACQ4,P0ACQ4,EXP,B,H,T,94,94,HB94T,32.01772,33.07836,1.06064,0.058855,0.008929,0.064364
1235,P0ACQ4-P0ACQ4-EXP-1i69-pdb1-B-0-A-0.pdb,P0ACQ4,P0ACQ4,EXP,B,H,S,94,94,HB94S,32.03946,33.82980,1.79034,0.049067,0.171640,0.160413
1233,P0ACQ4-P0ACQ4-EXP-1i69-pdb1-B-0-A-0.pdb,P0ACQ4,P0ACQ4,EXP,B,H,Q,94,94,HB94Q,32.03672,32.54062,0.50390,0.066331,0.180407,0.184684
1232,P0ACQ4-P0ACQ4-EXP-1i69-pdb1-B-0-A-0.pdb,P0ACQ4,P0ACQ4,EXP,B,H,P,94,94,HB94P,31.75970,34.52408,2.76438,0.053769,0.238179,0.245134


In [14]:
df = interface_df.copy()
df = df.drop(columns=["aa_wt", "aa_mt", "mut", "dG_wt", "dG_mt", "ddG", "dG_wt_sd", "dG_mt_sd", "ddG_sd"]).drop_duplicates()
df.prot_b.unique()

array(['P0ACQ4'], dtype=object)

In [15]:
from aledbmutil.gff import group_sequences

In [16]:
gene_interface_df = interface_df[((interface_df["prot_a"]=="P0ACQ4") & (interface_df["prot_b"]=="P0ACQ4"))]
# gene_interface_df = gene_interface_df["NC_000913v3_pos"].unique()
gene_interface_AAs = gene_interface_df["uniprot_pos"].unique()

# Outputting ranges for manual input into R
gene_interface_AAs.sort()
gene_interface_AA_ranges = group_sequences(gene_interface_AAs)
# gene_interface_AA_ranges

In [17]:
for rng_l in gene_interface_AA_ranges:
    feats_df = feats_df.append(
        {
            "feature": "OxyR subunit interface",
            "start": rng_l[0],
            "end": rng_l[-1],
        },
        ignore_index=True
    )

display(len(feats_df), feats_df.head())

/tmp/ipykernel_4653/526841779.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  feats_df = feats_df.append(
/tmp/ipykernel_4653/526841779.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  feats_df = feats_df.append(
/tmp/ipykernel_4653/526841779.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  feats_df = feats_df.append(
/tmp/ipykernel_4653/526841779.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  feats_df = feats_df.append(
/tmp/ipykernel_4653/526841779.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  feats_df = feats_df.append(
/tmp/

38

,feature,start,end,range
0,The C-terminal substrate-binding domain of the...,91,289,"(91, 199)"
1,Winged helix-like DNA-binding domain superfami...,1,84,"(1, 84)"
2,Periplasmic binding protein-like II,91,163,"(91, 73)"
3,Periplasmic binding protein-like II,268,284,"(268, 17)"
4,Periplasmic binding protein-like II,164,267,"(164, 104)"


In [18]:
feats_df.sort_values(by=["end", "start"], ascending=False, inplace=True)
feats_df.reset_index(drop=True, inplace=True)
feats_df["range"] = feats_df.apply(lambda r:_get_barh_broken_range(r), axis=1)  # barh_broken wants length for final value in range
display(len(feats_df), feats_df.head())

38

,feature,start,end,range
0,Chain,1,305,"(1, 305)"
1,TRANSCRIPTIONAL DUAL REGULATOR HCAR-RELATED,1,297,"(1, 297)"
2,HYDROGEN PEROXIDE-INDUCIBLE GENES ACTIVATOR,1,297,"(1, 297)"
3,"LysR, substrate-binding",89,293,"(89, 205)"
4,LysR substrate binding domain,89,293,"(89, 205)"


In [19]:
# The following entries are from interpro, though they are the CDD entries that I'm currently adding by hand
# https://www.ebi.ac.uk/interpro/protein/UniProt/P0ACQ4/
l = [
    {'feature':'OxyR subunit interface', 'start':103, 'end':103},
    {'feature':'OxyR subunit interface', 'start':106, 'end':107},
    {'feature':'OxyR subunit interface', 'start':110, 'end':110},
    {'feature':'OxyR subunit interface', 'start':114, 'end':114},
    {'feature':'OxyR subunit interface', 'start':122, 'end':126},
    {'feature':'OxyR subunit interface', 'start':219, 'end':219},
    {'feature':'OxyR subunit interface', 'start':222, 'end':223},
    {'feature':'OxyR subunit interface', 'start':226, 'end':226},
    {'feature':'OxyR subunit interface', 'start':229, 'end':230},
    {'feature':'OxyR subunit interface', 'start':232, 'end':233},
]

df = pd.DataFrame(l)
df["range"] = df.apply(lambda r:_get_barh_broken_range(r), axis=1)  # barh_broken wants length for final value in range
df = df.drop_duplicates()
df.head()

,feature,start,end,range
0,OxyR subunit interface,103,103,"(103, 1)"
1,OxyR subunit interface,106,107,"(106, 2)"
2,OxyR subunit interface,110,110,"(110, 1)"
3,OxyR subunit interface,114,114,"(114, 1)"
4,OxyR subunit interface,122,126,"(122, 5)"


In [20]:
feats_df = pd.concat([feats_df, df])

In [21]:
feats_df.to_pickle('./annot_df.pkl')